# Dynamic Convolution (Dynamic Conv / CondConv) — Tam Anlatım + Uygulamalı PyTorch

Bu notebook’ta **Dynamic Convolution** kavramını sıfırdan, adım adım öğreneceğiz ve **çalışan bir PyTorch implementasyonu** yazacağız.

## Bu notebook sonunda net öğrenmiş olacaksın
1) Dynamic Conv **neden var**? (static conv’un sınırı)
2) “Input’a göre filtre seçme / filtre karıştırma” fikri
3) **K adet uzman kernel** + **routing (gating)** ağı
4) Softmax ile ağırlıklandırılmış kernel birleşimi
5) PyTorch’ta iki implementasyon:
   - (A) Anlaşılır: per-sample ağırlık hesapla + kernel oluştur (eğitim amaçlı)
   - (B) Daha verimli: grouped-conv numarası (opsiyonel)
6) DynamicConv kullanan mini model (DynamicNet)

Not: Dynamic Conv güçlüdür ama **maliyet ve karmaşıklık** getirir. Bu notebook’ta bunun trade-off’larını da açıkça konuşacağız.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(0)
print('torch:', torch.__version__)


torch: 2.9.1+cpu


---
## 1) Motivasyon: Neden Dynamic Conv?

Normal (static) `Conv2d` şudur:
- Ağırlıklar **sabit** (`W`) ve her input için aynı filtre uygulanır.

Bu iyi bir inductive bias’tır ama bir sınırlama getirir:
- Farklı görüntüler / farklı içerikler için aynı filtre setini zorunlu kullanırsın.

Dynamic Conv fikri:
> Filtreleri input’a göre **dinamik** hale getir.

Yani model şunu yapar:
- Input’tan bir özet çıkarır (global context)
- Bu özetle **hangi kernel(ler)** daha uygun karar verir
- K adet kernel’in ağırlıklı karışımıyla “o anki” conv filtresini üretir

Bu yaklaşımın iki tipik ismi:
- **CondConv (Conditional Convolution)**: koşullu kernel kombinasyonu
- **Dynamic Conv**: genel isim


---
## 2) Temel Formül (K uzman kernel)

Elimizde K adet uzman kernel olsun:
- `W_1, W_2, ..., W_K`

Routing (gating) ağı, input x için K boyutlu bir ağırlık üretsin:
- `a(x) = [a_1, ..., a_K]`
- Genellikle **softmax** ile normalize edilir: `sum a_k = 1`

Dinamik kernel:

$W(x) = \sum_{k=1}^{K} a_k(x)\, W_k$

Sonra conv:

$y = \mathrm{Conv}(x, W(x))$

Özet:
- Kernel sayısı K kadar artar
- Ama her örnek için **tek bir efektif kernel** çalıştırırsın
- Ek maliyet: routing ağı + kernel kombinasyonu


---
## 3) Routing Ağı (Gating) Ne Kullanır?

Pratikte gating çoğu zaman şu şekilde yapılır:
1) Global Average Pooling (GAP): `x -> v` (B, C)
2) Küçük bir MLP (Linear → ReLU → Linear)
3) Softmax ile K ağırlık

Bu, SE (Squeeze-Excitation) mantığına benzer ama çıkış **kanal ağırlığı** değil, **kernel karışımı ağırlığıdır**.


---
## 4) DynamicConv2d — Eğitim Odaklı, Anlaşılır Implementasyon

Bu versiyonun amacı **okunabilirlik**:
- Her örnek için `a_k` hesaplanır
- Kernel ağırlıklı toplanır
- Sonra conv uygulanır


In [2]:
class DynamicConv2d(nn.Module):
    """Dynamic/Conditional Conv2d (CondConv tarzı)

    K adet uzman kernel saklar.
    Her input için gating ağı K ağırlık üretir.
    Kernel'ler ağırlıklı toplanıp efektif kernel ile conv yapılır.

    Not: Bu notebook'ta anlaşılır olması için per-sample conv döngüsü kullanıyoruz.
    """
    def __init__(self, cin, cout, k=3, stride=1, padding=1, dilation=1,
                 K=4, reduction=4, bias=False, temperature=1.0):
        super().__init__()
        self.cin = cin
        self.cout = cout
        self.k = k
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.K = K
        self.temperature = temperature

        # Uzman kernel bankası: (K, cout, cin, k, k)
        self.weight = nn.Parameter(torch.randn(K, cout, cin, k, k) * 0.02)
        self.bias = None
        if bias:
            self.bias = nn.Parameter(torch.zeros(K, cout))

        # Routing / gating: GAP -> MLP -> K logits
        hidden = max(1, cin // reduction)
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.fc1 = nn.Linear(cin, hidden)
        self.fc2 = nn.Linear(hidden, K)

    def routing(self, x):
        # x: (B, C, H, W) -> a: (B, K)
        v = self.gap(x).flatten(1)         # (B, C)
        h = F.relu(self.fc1(v))           # (B, hidden)
        logits = self.fc2(h)              # (B, K)
        a = F.softmax(logits / self.temperature, dim=1)
        return a

    def forward(self, x):
        B, C, H, W = x.shape
        a = self.routing(x)  # (B, K)

        # Dinamik kernel üretimi: Wdyn (B, cout, cin, k, k)
        Wdyn = torch.einsum('bk,kocij->bocij', a, self.weight)

        # (Opsiyonel) bias
        bdyn = None
        if self.bias is not None:
            bdyn = torch.einsum('bk,kc->bc', a, self.bias)  # (B, cout)

        # Per-sample conv (anlaşılır versiyon)
        outs = []
        for i in range(B):
            yi = F.conv2d(
                x[i:i+1],
                Wdyn[i],
                bias=None if bdyn is None else bdyn[i],
                stride=self.stride,
                padding=self.padding,
                dilation=self.dilation,
            )
            outs.append(yi)
        return torch.cat(outs, dim=0)


### 4.1) Hızlı Test
- Giriş shape
- Çıkış shape
- Routing ağırlıkları toplamı 1 mi?


In [3]:
x = torch.randn(2, 16, 32, 32)
dyn = DynamicConv2d(cin=16, cout=32, k=3, padding=1, K=4, reduction=4)
y = dyn(x)

a = dyn.routing(x)
print('x:', x.shape)
print('y:', y.shape)
print('routing a:', a.shape)
print('row sums (should be 1):', a.sum(dim=1))


x: torch.Size([2, 16, 32, 32])
y: torch.Size([2, 32, 32, 32])
routing a: torch.Size([2, 4])
row sums (should be 1): tensor([1., 1.], grad_fn=<SumBackward1>)


---
## 5) DynamicConvBlock: DynamicConv + BN + ReLU

Modelde tek katman değil, blok halinde kullanmak daha mantıklı.
Burada:
- DynamicConv2d
- BatchNorm
- ReLU

şeklinde bir blok tanımlıyoruz.


In [4]:
class DynamicConvBlock(nn.Module):
    def __init__(self, cin, cout, k=3, stride=1, padding=1, K=4, reduction=4, temperature=1.0):
        super().__init__()
        self.dyn = DynamicConv2d(cin, cout, k=k, stride=stride, padding=padding, K=K, reduction=reduction, temperature=temperature)
        self.bn = nn.BatchNorm2d(cout)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.dyn(x)
        x = self.bn(x)
        x = self.act(x)
        return x


---
## 6) Tam Mini Model: DynamicNet

Bu model:
- Stem (normal conv)
- 3 stage (DynamicConvBlock)
- Head (GAP + FC)

Amaç: Dynamic Conv’u model seviyesinde görmek.


In [5]:
class DynamicNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=10, K=4, temperature=1.0):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(in_channels, 32, 3, padding=1, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
        )

        self.stage1 = DynamicConvBlock(32, 64,  k=3, stride=2, padding=1, K=K, reduction=4, temperature=temperature)
        self.stage2 = DynamicConvBlock(64, 128, k=3, stride=2, padding=1, K=K, reduction=4, temperature=temperature)
        self.stage3 = DynamicConvBlock(128, 256, k=3, stride=2, padding=1, K=K, reduction=4, temperature=temperature)

        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, num_classes)

    def forward(self, x, verbose=False):
        if verbose: print('input:', x.shape)
        x = self.stem(x)
        if verbose: print('stem :', x.shape)
        x = self.stage1(x)
        if verbose: print('s1   :', x.shape)
        x = self.stage2(x)
        if verbose: print('s2   :', x.shape)
        x = self.stage3(x)
        if verbose: print('s3   :', x.shape)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        out = self.fc(x)
        if verbose: print('out  :', out.shape)
        return out


In [6]:
model = DynamicNet(in_channels=3, num_classes=10, K=4, temperature=1.0)
x = torch.randn(2, 3, 64, 64)
y = model(x, verbose=True)
print('\nfinal:', y.shape)


input: torch.Size([2, 3, 64, 64])
stem : torch.Size([2, 32, 64, 64])
s1   : torch.Size([2, 64, 32, 32])
s2   : torch.Size([2, 128, 16, 16])
s3   : torch.Size([2, 256, 8, 8])
out  : torch.Size([2, 10])

final: torch.Size([2, 10])


---
## 7) Dynamic Conv’un Artıları / Eksileri (Gerçekçi)

### Artılar
- Input’a göre adaptasyon → farklı örneklere farklı filtre davranışı
- Mixture-of-Experts sezgisi (uzman kernel bankası)

### Eksiler
- Ek compute: routing ağı + kernel kombinasyonu
- Kod karmaşıklığı (özellikle batch üzerinde verimli uygulama)
- Bazı durumlarda kararsız eğitim (temperature, regularization gerekebilir)

### Pratik ipuçları
- `K` küçük başla: 4 veya 8
- `temperature` ile softmax keskinliğini kontrol et
  - düşük T → daha seçici
  - yüksek T → daha karışım
- Routing’e dropout / weight decay eklemek bazen iyi gelir


---
## 8) (Opsiyonel) Daha Verimli Uygulama Ne Demek?

Bu notebook’ta anlaşılır olması için per-sample döngü kullandık.
Gerçek projede batch’i tek seferde işlemek için:
- grouped-conv trick
- top-k routing
- dinamikliği sadece 1×1 conv’da kullanma

gibi optimizasyonlar yapılır.
